In [ ]:
import random
from random import randint
import json
from datetime import datetime
from operator import add
import time
from itertools import chain
from itertools import combinations, groupby
from operator import add
from collections import Counter
import itertools
import json
import math
import random
import sys
import time
from functools import reduce
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
data_path = "/content/drive/MyDrive/DSCI553/project/data"

In [ ]:
def getFeatures(data_path): 
    user_path = data_path+"/user.json"

    # load the user and business features
    user_features = {} 
    with open(user_path) as file: 
        for l in file.readlines(): 
            data = json.loads(l)
            user_features[data["user_id"]]=[float(data["review_count"]),float(data["useful"]), float(data["average_stars"])]

    business_path = data_path+"/business.json"
    business_features = {}
    with open(business_path) as file: 
        for l in file.readlines(): 
            data = json.loads(l)
            business_features[data["business_id"]] = [float(data["stars"]), float(data["review_count"]), float(data["is_open"])]

    photo_path = data_path+"/photo.json"
    photo_features = {}
    with open(photo_path) as file: 
        for l in file.readlines(): 
            data = json.loads(l)
            business_id = data["business_id"]
            if business_id not in photo_features: 
                photo_features[business_id]=1.0
            else: 
                photo_features[business_id]+=1.0

    tip_path = data_path+"/tip.json"
    tip_features = {}
    with open(tip_path) as file: 
        for l in file.readlines():
            data = json.loads(l)
            b_id = data["business_id"]
            u_id = data["user_id"]
            if (u_id, b_id) in tip_features: 
                tip_features[(u_id, b_id)]+=1.0
            else: 
                tip_features[(u_id, b_id)]=1.0
    return user_features, business_features, photo_features, tip_features


In [ ]:
user_path = data_path+"/user.json"

# load the user and business features
user_features = {} 
with open(user_path) as file: 
    for l in file.readlines(): 
        data = json.loads(l)
        user_features[data["user_id"]]=[float(data["review_count"]),float(data["useful"]), float(data["average_stars"])]

In [ ]:
business_path = data_path+"/business.json"
business_features = {}
with open(business_path) as file: 
    for l in file.readlines(): 
        data = json.loads(l)
        business_features[data["business_id"]] = [float(data["stars"]), float(data["review_count"]), float(data["is_open"])]

In [ ]:
photo_path = data_path+"/photo.json"
photo_features = {}
with open(photo_path) as file: 
    for l in file.readlines(): 
        data = json.loads(l)
        business_id = data["business_id"]
        if business_id not in photo_features: 
            photo_features[business_id]=1.0
        else: 
            photo_features[business_id]+=1.0


In [ ]:
tip_path = data_path+"/tip.json"
tip_features = {}
with open(tip_path) as file: 
    for l in file.readlines():
        data = json.loads(l)
        b_id = data["business_id"]
        u_id = data["user_id"]
        if (u_id, b_id) in tip_features: 
            tip_features[(u_id, b_id)]+=1.0
        else: 
            tip_features[(u_id, b_id)]=1.0


In [ ]:
# get the business/user pair and the ground truth
def getData(path): 
    train_path = path
    train = []
    with open (train_path) as file: 
        for l in file.readlines(): 
            data = l.rstrip().split(",")
            if data[0]!="user_id": 
                user_id = data[0]
                business_id = data[1]
                u_f = user_features[user_id]
                b_f = business_features[business_id]
                try: 
                    photo = [photo_features[business_id]]
                except: 
                    photo = [0.0]
                try: 
                    tip = [tip_features[(user_id, business_id)]]
                except: 
                    tip = [0.0]
              
                train.append(data+u_f+b_f+photo+tip)
    return train


In [ ]:
train_path = data_path+"/yelp_train.csv"
train = getData(train_path)
train_x = np.array([x[3:] for x in train])
train_y = np.array([float(x[2]) for x in train])

In [ ]:
test_path = data_path+"/yelp_val.csv"
test = getData(test_path)

In [ ]:
test_x = np.array([x[3:] for x in test])
test_y = np.array([float(x[2]) for x in test])

In [ ]:
d_train = xgb.DMatrix(train_x, label=train_y)
d_test = xgb.DMatrix(test_x, label=test_y)

In [ ]:
params ={
    'max_depth':6,
    'min_child_weight': 2,
    'eta':.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    "eval_metric" :"rmse",
    'objective': 'reg:linear',
}
num_boost_round = 500
early_stopping_rounds = 10
model = xgb.train(params, 
                  d_train, 
                  num_boost_round=1000, 
                  evals=[(d_test, "Test")], 
                  early_stopping_rounds=20)

[22:09:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-rmse:3.28268
Will train until Test-rmse hasn't improved in 20 rounds.
[1]	Test-rmse:3.13513
[2]	Test-rmse:2.99422
[3]	Test-rmse:2.86256
[4]	Test-rmse:2.73841
[5]	Test-rmse:2.62099
[6]	Test-rmse:2.50886
[7]	Test-rmse:2.40452
[8]	Test-rmse:2.3063
[9]	Test-rmse:2.21238
[10]	Test-rmse:2.12532
[11]	Test-rmse:2.04354
[12]	Test-rmse:1.96549
[13]	Test-rmse:1.89236
[14]	Test-rmse:1.82383
[15]	Test-rmse:1.76081
[16]	Test-rmse:1.70064
[17]	Test-rmse:1.64456
[18]	Test-rmse:1.59299
[19]	Test-rmse:1.54507
[20]	Test-rmse:1.50032
[21]	Test-rmse:1.45887
[22]	Test-rmse:1.42042
[23]	Test-rmse:1.38383
[24]	Test-rmse:1.35003
[25]	Test-rmse:1.31877
[26]	Test-rmse:1.28991
[27]	Test-rmse:1.26369
[28]	Test-rmse:1.23973
[29]	Test-rmse:1.21706
[30]	Test-rmse:1.19629
[31]	Test-rmse:1.17718
[32]	Test-rmse:1.15968
[33]	Test-rmse:1.14364
[34]	Test-rmse:1.12924
[35]	Test-rmse:1.11609

In [ ]:
pre = model.predict(xgb.DMatrix(test_x))

In [ ]:
error = mean_squared_error(test_y, pre, squared=False)

In [ ]:
error

0.9786676828144032

In [ ]:
model.save_model("model_{:.6f}.txt".format(error))

In [ ]:
# load model 
test_model = xgb.Booster()
test_model.load_model("/content/model_0.978668.json")
pre_test = test_model.predict(xgb.DMatrix(test_x))
error_test = mean_squared_error(test_y, pre_test, squared=False)

In [ ]:
def getPrediction(test, model_path):
    


In [ ]:
p = getPrediction(test, )

In [ ]:
test[0]

['wf1GqnKQuvH-V3QN80UOOQ',
 'fThrN4tfupIGetkrz18JOg',
 '5.0',
 118.0,
 2.0,
 4.02,
 3.5,
 122.0,
 1.0,
 11.0,
 0.0]

In [ ]:
test = np.array(test)

In [ ]:
test_inde = test[:,:2]
test_data = test[:,-8:]

In [ ]:
def getPrediction(test, model_path): 
    test_inde = test[:,:2]
    test_data = test[:,-8:]
    model = xgb.Booster()
    model.load_model(model_path) 
    prediction = model.predict(xgb.DMatrix(test_data))
    return test_inde, prediction

In [ ]:
model_path = "/content/model_0.978668.json"
index, pre = getPrediction(test, model_path)

In [ ]:
pre

array([3.794866 , 4.8734784, 4.866691 , ..., 3.6528099, 4.022067 ,
       3.4679842], dtype=float32)

In [ ]:
xgb.__version__

'0.90'

In [ ]:
import pickle
pickle.dump(model, open("pima1.pickle.dat", "wb"))

In [ ]:
loaded_model = pickle.load(open("pima.pickle.dat", "rb"))

In [ ]:
loaded_model

In [ ]:
params ={
    'max_depth':6,
    'min_child_weight': 2,
    'eta':.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    "eval_metric" :"rmse",
    'objective': 'reg:linear',
}